# Домашнее задание № 10. Генерация текста

### Задание 1 (10 баллов).

В семинаре мы работали с датасетами инструкций alpaca и dolly. Они англоязычные. В домашке вам нужно создать аналогичный датасет на русском языке и обучить аналогичную модель на этом датасете.
В качестве итогового результата у вас должна получится модель, которая может связно отвечать на русскоязычные инструкции на русском языке. Приведите как минимум три разных примера. Правильность ответов не так важна, так как вы скорее всего будете использовать небольшие модели, но текст должен быть не рандомным.

Русскоязычный датасет инструкций должен быть больше 5 тысяч примеров. Он может быть основнован на alpaca/dolly (например, вы можете просто прогнать все через переводную модель, которая была на семинаре, или даже google translate). Или вы можете придумать способ создать аналогичный датасет каким-то другим способом (переделать открытые датасеты с помощью правил). Датасет может быть не уникальным, можно скооперироваться с одногруппниками и сделать один датасет на всех.

Вы можете попробовать дообучать любую небольшую decoder-only модель. Скорее всего лучше всего будут работать модели, изначально обученные на русском языке (rugpt например). Но возможно даже модели вроде opt можно будет дообучить на русскоязычных инструкциях.

Это задание гораздно менее определенное, по сравнению с предыдущими. Поэтому не стесняйтесь задавать дополнительные вопросы в чате или лично, если у вас возникнут трудности.

In [1]:
!pip install datasets deep_translator zstandard jsonlines transformers

In [2]:
!pip install accelerate -U

In [20]:
import pandas as pd
import numpy as np
import torch
import json
from datasets import load_dataset

import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer
import accelerate

In [4]:
accelerate.__version__

'0.29.3'

Берём русскоязычную Альпаку.

In [5]:
ru_alpaca = load_dataset("IlyaGusev/ru_turbo_alpaca")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for IlyaGusev/ru_turbo_alpaca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/ru_turbo_alpaca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `d

In [6]:
ru_alpaca

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'],
        num_rows: 29822
    })
})

In [66]:
ru_alpaca["train"][433]["instruction"]

'Даны два списка чисел. Верните список, состоящий из уникальных чисел, которые присутствуют только в первом списке.'

In [7]:
ru_alpaca["train"][433]["input"]

'[1, 2, 3, 4, 5], [3, 4, 5, 6, 7]'

In [8]:
ru_alpaca["train"][433]["output"]

'[1, 2]'

In [9]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

In [10]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [11]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

Немного модифицируем код, чтобы он смог переварить русскоязычную Альпаку.

In [12]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = data["train"]

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [13]:
model_name = "facebook/opt-125m"
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        max_length=512,
        cache_dir="huggingface_cache",
    )

In [14]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    cache_dir="huggingface_cache",
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

In [15]:
train_dataset = SupervisedDataset(tokenizer=tokenizer, data = ru_alpaca)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [16]:
train_args = transformers.TrainingArguments(learning_rate=1e-5,
                 num_train_epochs=1,
                 per_device_train_batch_size=2,
                 gradient_accumulation_steps=1,
                 evaluation_strategy='no',
                 weight_decay=0.,
                 warmup_ratio=0.03,
                 lr_scheduler_type="cosine",
                 save_strategy='no',
                 logging_steps=1000,
                 output_dir="opt125_instruct_ft")

In [17]:
trainer = Trainer(model=model,
                 tokenizer=tokenizer,
                 args=train_args,
                 train_dataset=train_dataset,
                 eval_dataset=None,
                 data_collator=data_collator)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [18]:
trainer.train()

Step,Training Loss
1000,1.268900
2000,1.176900
3000,1.114800
4000,1.095700
5000,1.073800
6000,1.045500
7000,1.032300
8000,1.017200
9000,1.001100
10000,0.993500


TrainOutput(global_step=14911, training_loss=1.0486835374589754, metrics={'train_runtime': 4163.255, 'train_samples_per_second': 7.163, 'train_steps_per_second': 3.582, 'total_flos': 6836609053440000.0, 'train_loss': 1.0486835374589754, 'epoch': 1.0})

In [19]:
trainer.save_model('opt125_ft_02')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


In [69]:
MODEL_NAME = 'opt125_ft_02'

In [70]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512, max_length=512)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, max_length=512)

In [191]:
def predict_for_instruction(instruction, text, model):
    text = text.replace('\n', ' ')
    prompt = ("Below is an instruction that describes a task, paired with an input that provides further context. "
              "Write a response that appropriately completes the request.\n\n"
              f"### Instruction:\n{instruction}\n\n### Input:\n{text}\n\n### Response:")

    inputs = tokenizer([prompt],
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        temperature=0.4,
        max_length=1000,
        no_repeat_ngram_size=15,
        repetition_penalty= 0.5,
        early_stopping=False,
        num_return_sequences= 1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=True,
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs[0]):], skip_special_tokens=False)
    return summaries[0]

Модель понимает, когда мы просим её продолжить историю и пытается подобрать подходящие слова.

In [220]:
# заголовок одного из моих текстов
text = """
 Эксперты РАН представили три сценария глобальных торговых войн
"""

In [188]:
instruction = "Продолжи следующий текст"
predict_for_instruction(instruction, text, model)

'Эксперты РАг представили текст в глобальной целой глобальной торговой сцене. Это представило вредную природу, представило в глобальной сцене в глобальной тохнологии, представило представило подготовку в глобальной реце в глобальной глобальной сходе. Это предстательно представило более ходить в глобальной упражнении в глобальной яйцах, представило быстро в глобальной природе в глобальной, представило достижение природы в глобальной'

In [246]:
text = """
Им необходимы специальные меры поддержки, направленные на укрепление конкурентоспособных
"""


In [247]:
instruction = "Допиши"
predict_for_instruction(instruction, text, model)

'Им необходимы поддержки, которые можно поддерживать на проблему конкурентоспобение конкурентопобенипа конкурентоспательному конкурентосбенность. Им необходимо поддерживательно поддерживавать на проблетку конкурентос�бенному конкуренту конкурентосженипа конкуреентопобенипа поддержки, как конкурентосбиродные поддержки можно поддерпить конкурентосбуржные поддержки, компьцы поддержки модели поддерживать свои поддержки конкурентосбанных специальных специалипатипом.</s>'

Также она понимает, когда нам нужно сократить текст, и переписывает его короче.

In [243]:
# комментарий эксперта
text = """
 Введение санкций показало, что они наносят ущерб не только и даже не столько странам, против которых они направлены, но и странам, которые вводят эти ограничения. Наиболее показательным является пример Евросоюза, который испытывает экономические проблемы, а также социальную напряжённость, введя санкции против России. Наращивание оборотов в торговом противостоянии представляется нелогичным, тем более когда участниками торговых войн являются крупнейшие экономики мира. В этой связи, более вероятным представляется второй сценарий. В продолжение этой мысли глобальная торговая война скорее останется на бумаге и не будет реализована в действительности.
"""
len(text)

656

In [244]:
instruction = "Сократи"
shorter = predict_for_instruction(instruction, text, model)
print(shorter)
len(shorter)

Введение санкшций показал не только и делается не столько столько и даже столько страхов и даже не стоит не произведение. В процессе вы можете показать несколько дней и показать нестановиться на против которые и показать на прот


228

In [232]:
text = """
Подход, предлагаемый министерством экономического развития, основан на принципах, давно нашедших подтверждение в международном опыте. Закон необходим. Хотя конкретные параметры и могут варьироваться в зависимости от изменяющихся условий и обратной связи от бизнеса, и в любом случае потребуют регулярной донастройки в будущем. Например, в ЕС в этом году примерно на 25% повышают планку по доходам и размеру активов для отнесения компаний к микро, малому и среднему бизнесу в целях отражения накопленной за последние годы инфляции при этом сохраняя критерии по численности.
"""
len(text)

574

In [234]:
instruction = "Перепиши этот текст короче"
shorter = predict_for_instruction(instruction, text, model)
print(shorter)
len(shorter)

Подход, предложение министерством министерством, предлагаемые министерства подходят при подходят подходят под выбором и предлагаемые экономические развития. Подход подходят при этом сочетает подходят пользовательный программ и предлагает при подходящие выборы и предлагает регуля


279

Ещё модель понимает, когда мы просим её провести математические вычсления.

(Получается так же хорошо, как у меня - гордое гуманитарное образование)

In [217]:
text = "8+5"

In [219]:
instruction = "Посчитай"
predict_for_instruction(instruction, text, model)

'8</s>'

In [248]:
text = "(82+1)*2"

In [242]:
instruction = "Вычисли"
predict_for_instruction(instruction, text, model)

'2</s>'

Кажется, модель лучше справляется с задачами, если получает более короткие инструкции. Сравните следующий пример с двумя выше.

In [249]:
text = "(82+1)*2"

In [250]:
instruction = "Найди результат следующего выражения"
predict_for_instruction(instruction, text, model)

'Найди резулятат следующее выражение:\n1. Найди в горы в горы в горы с вещестные вещи.\n2. Найди в гори в горы в горизонте в горы в горы.\n1. Найди в гулятый гор в горы в горы, в горы в горые в горы в гористый горы.\n2. Найди в жизнь в горы в горый горы в горы в старый горы в гостей.\n1. Найди в жижный горы в горащестный горы в государственный горы.\n2. Независит в горы в горы: в горы в горыстый горы в гористей.\n1. Найжит в горы в госудаственный горы в готонте.\n2. Найди в вызыванный горы в гулятый гостей.\n1. Независит в старый горащественный горы, в горы загрузит в горы в готонтте.\n2. Найди загрузит в вызыванныго горащественности в горы в горащий горы.\n1. Найжит горы в горы вызыванный жизнь в горы.\n2. Найжит в грузит в горы, в горащественную горы в горы важности.\n1. Найжит загрузит в жизнь в гостей.\n2. Найжит зазверит в горы в гощ'